In [1]:
import pandas as pd
import pareto
import os
import sys
from itertools import chain

sys.path.append("..")

In [2]:
import rbf_functions

# Load solutions for each RBF for all seeds:

In [21]:
rbfs = [
    rbf_functions.squared_exponential_rbf,
    rbf_functions.original_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

pareto_sets = {}
for entry in rbfs:
    sollist = []
    solutions = []
    name = entry.__name__
    output_dir = f"../output/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith("solution.csv"):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            solutions.append(df_temp.values.tolist())

    pareto_sets[name] = list(chain.from_iterable(solutions))

pareto_sets.keys()
print("Amount of solutions for each RBF:")
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
# print(f"Loaded into list 'solutions': {', '.join(sollist)}")

Amount of solutions for each RBF:
squared_exponential_rbf 14323
original_rbf 15650
inverse_quadratic_rbf 9372
inverse_multiquadric_rbf 10593
exponential_rbf 7753
matern32_rbf 13423
matern52_rbf 9339


In [24]:
rbfs = [
    rbf_functions.squared_exponential_rbf,
    rbf_functions.original_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

pareto_sets = {}
for entry in rbfs:
    sets_per_seed = []
    name = entry.__name__
    output_dir = f"../output/{name}/"

    for seed in np.arange(10, 110, 10):
        solutions = pd.read_csv(os.path.join(output_dir, f"{seed}_solution.csv"))
        variables = pd.read_csv(
            os.path.join(output_dir, f"{seed}_variables.csv"), header=None
        )

        combined = pd.concat([variables, solutions], axis=1)
        sets_per_seed.append(combined)

    pareto_sets[name] = pd.concat(sets_per_seed)


print("Amount of solutions for each RBF:")
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))

Amount of solutions for each RBF:
squared_exponential_rbf 14323
original_rbf 15650
inverse_quadratic_rbf 9372
inverse_multiquadric_rbf 10593
exponential_rbf 7753
matern32_rbf 13423
matern52_rbf 9339


# Create reference set for each RBF, and save:

In [39]:
# this code produces the reference set per RBF, but in the stored results also included the associated decision variables

reference_sets = {}
for rbf in pareto_sets:
    data = pareto_sets[rbf]

    print(rbf, len(data))
    nondominated = pareto.eps_sort(
        [data.values],
        [32, 33, 34, 35, 36, 37],
        [0.5, 0.05, 0.05, 0.05, 0.001, 0.05],
        maximize=[32, 33, 34, 35, 37],
    )
    reference_sets[rbf] = nondominated
    df_nondom = pd.DataFrame(nondominated, columns=data.columns)
    print(rbf, len(nondominated))
    df_nondom.to_csv(
        f"./refsets/{rbf}_refset_with_variables.csv", index=False, header=True
    )

squared_exponential_rbf 14323
squared_exponential_rbf 2142
original_rbf 15650
original_rbf 2675
inverse_quadratic_rbf 9372
inverse_quadratic_rbf 1684
inverse_multiquadric_rbf 10593
inverse_multiquadric_rbf 1636
exponential_rbf 7753
exponential_rbf 1585
matern32_rbf 13423
matern32_rbf 2268
matern52_rbf 9339
matern52_rbf 1774


In [12]:
reference_sets = {}
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
    nondominated = pareto.eps_sort(
        [pareto_sets[rbf]],
        [0, 1, 2, 3, 4, 5],
        [0.5, 0.05, 0.05, 0.05, 0.001, 0.05],
        maximize=[0, 1, 2, 3, 5],
    )
    reference_sets[rbf] = nondominated
    df_nondom = pd.DataFrame(
        nondominated,
        columns=[
            "hydropower",
            "atomicpowerplant",
            "baltimore",
            "chester",
            "environment",
            "recreation",
        ],
    )
    print(rbf, len(df_nondom))
    df_nondom.to_csv(f"../notebooks/refsets/{rbf}_refset.csv", index=False, header=True)

squared_exponential_rbf 14323
squared_exponential_rbf 2142
original_rbf 15650
original_rbf 2675
inverse_quadratic_rbf 9372
inverse_quadratic_rbf 1684
inverse_multiquadric_rbf 10593
inverse_multiquadric_rbf 1636
exponential_rbf 7753
exponential_rbf 1585
matern32_rbf 13423
matern32_rbf 2268
matern52_rbf 9339
matern52_rbf 1774


# Find decision variables that belong to the generated refset:

In [5]:
d_refvar = {}
d_refsol = {}
for entry in rbfs:
    name = entry.__name__
    df_sol = pd.DataFrame(
        columns=[
            "hydropower",
            "atomicpowerplant",
            "baltimore",
            "chester",
            "environment",
            "recreation",
        ]
    )
    df_var = pd.DataFrame(
        columns=[
            0,
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            19,
            20,
            21,
            22,
            23,
            24,
            25,
            26,
            27,
            28,
            29,
            30,
            31,
        ]
    )
    output_dir = f"../output/{name}"

    for filename in os.listdir(output_dir):
        # load
        if filename.endswith("solution.csv"):
            df_ts = pd.read_csv(f"{output_dir}/{filename}", header=0)
            df_sol = df_sol.append(df_ts, ignore_index=True)
        elif filename.endswith("variables.csv"):
            df_tv = pd.read_csv(f"{output_dir}/{filename}", header=None)
            df_var = df_var.append(df_tv, ignore_index=True)

    sol = df_sol.values.tolist()
    var = df_var.values.tolist()
    ref = reference_sets[name]
    refvar = []
    refsol = []

    for idx, value in enumerate(sol):
        if value in ref:
            refvar.append(var[idx])
            refsol.append(sol[idx])
    d_refsol[name] = pd.DataFrame(
        refsol,
        columns=[
            "hydropower",
            "atomicpowerplant",
            "baltimore",
            "chester",
            "environment",
            "recreation",
        ],
    )
    d_refvar[name] = pd.DataFrame(refvar)
    d_refsol[name].to_csv(
        f"../notebooks/refsets/{name}_refset.csv", index=False, header=True
    )
    d_refvar[name].to_csv(
        f"../notebooks/refsets/{name}_refset_variables.csv", index=False, header=False
    )

/var/folders/fg/q50zvs055fg3jrm35194kn5dsywm3z/T/ipykernel_6897/2561762121.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_var = df_var.append(df_tv, ignore_index=True)
/var/folders/fg/q50zvs055fg3jrm35194kn5dsywm3z/T/ipykernel_6897/2561762121.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sol = df_sol.append(df_ts, ignore_index=True)
/var/folders/fg/q50zvs055fg3jrm35194kn5dsywm3z/T/ipykernel_6897/2561762121.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_var = df_var.append(df_tv, ignore_index=True)
/var/folders/fg/q50zvs055fg3jrm35194kn5dsywm3z/T/ipykernel_6897/2561762121.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [13]:
for entry in rbfs:
    name = entry.__name__
    print(name)
    print(f"refset: {len(d_refsol[name])}")
    print(f"varset: {len(d_refvar[name])}")

squared_exponential_rbf
refset: 2142
varset: 2142
original_rbf
refset: 2675
varset: 2675
inverse_quadratic_rbf
refset: 1684
varset: 1684
inverse_multiquadric_rbf
refset: 1636
varset: 1636
exponential_rbf
refset: 1585
varset: 1585
matern32_rbf
refset: 2268
varset: 2268
matern52_rbf
refset: 1774
varset: 1774


# Generate global reference set for all RBFs:

In [14]:
x = 0
for rbf in pareto_sets:
    x += len(pareto_sets[rbf])
    print(rbf, len(pareto_sets[rbf]))
print("total:", x)

pareto_set = {}
sollist = []
solutions = []
for entry in rbfs:
    name = entry.__name__
    output_dir = f"../output/{name}/"
    for filename in os.listdir(output_dir):
        if filename.endswith("solution.csv"):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}{filename}", header=0)
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

squared_exponential_rbf 14323
original_rbf 15650
inverse_quadratic_rbf 9372
inverse_multiquadric_rbf 10593
exponential_rbf 7753
matern32_rbf 13423
matern52_rbf 9339
total: 80453


80453

In [16]:
print(len(pareto_set))
nondominated = pareto.eps_sort(
    [pareto_set],
    [0, 1, 2, 3, 4, 5],
    [0.5, 0.05, 0.05, 0.05, 0.001, 0.05],
    maximize=[0, 1, 2, 3, 5],
)
df_nondom = pd.DataFrame(
    nondominated,
    columns=[
        "hydropower",
        "atomicpowerplant",
        "baltimore",
        "chester",
        "environment",
        "recreation",
    ],
)
print(len(nondominated))
df_nondom.to_csv(f"../notebooks/refsets/global_refset.csv", index=False, header=True)

80453
2859
